In [1]:
import os
from pinecone import Pinecone
from groq import Groq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate


d:\ANACONDA\envs\globalenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_FOLDER = r"D:\RUFUS\data"

INDEX_NAME = "mayihelp"      # existing index
NAMESPACE = "rufus-data"

TOP_K = 5
MAX_CONTEXT_CHARS = 8000
GROQ_MODEL = "llama-3.1-8b-instant"
BATCH_SIZE = 50


In [3]:
def load_text_files(folder):
    texts = []
    for file in os.listdir(folder):
        if file.endswith(".txt"):
            with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
                content = f.read().strip()
                if content:
                    texts.append(content)
    return texts


documents = load_text_files(DATA_FOLDER)

if not documents:
    raise RuntimeError("No .txt files found in data folder")

print(f" Loaded {len(documents)} text files")


 Loaded 5 text files


In [4]:
documents

['MMTT\nVHF / UHF and GSM Hybrid Handheld Device\nSmart Monitoring and Tracking Transceiver\n\n(Place line-art device illustration in the same position as reference)\n\nMMTT\nUser Manual\n\nLogos and Marks\n\nHybrid Tracking\nOffline and Online Communication\nSOS Enabled\nIP-Rated Design\n\nTeam PlayMetrics\n\nNOTICE\n\nLightning and Environmental Safety\n\nNever operate the MMTT handheld device outdoors during thunderstorms or in high-risk lightning environments. The antenna and the user may form a conductive path that increases the risk of injury.\n\nIf severe weather approaches, switch off the device, move to a safe enclosed area, and avoid elevated or open locations.\n\nTeam PlayMetrics is not responsible for damage or injury caused by unsafe usage conditions.\n\nIntroduction\n\nThis manual describes the standard features and basic operation of the MMTT Hybrid Tracking Transceiver, developed by Team PlayMetrics for mission-critical communication and tracking.\n\nAdvanced configurat

In [5]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = splitter.split_text("\n\n".join(documents))
print(f" Created {len(chunks)} chunks")


 Created 86 chunks


In [6]:
chunks

['MMTT\nVHF / UHF and GSM Hybrid Handheld Device\nSmart Monitoring and Tracking Transceiver\n\n(Place line-art device illustration in the same position as reference)\n\nMMTT\nUser Manual\n\nLogos and Marks\n\nHybrid Tracking\nOffline and Online Communication\nSOS Enabled\nIP-Rated Design\n\nTeam PlayMetrics\n\nNOTICE\n\nLightning and Environmental Safety',
 'Team PlayMetrics\n\nNOTICE\n\nLightning and Environmental Safety\n\nNever operate the MMTT handheld device outdoors during thunderstorms or in high-risk lightning environments. The antenna and the user may form a conductive path that increases the risk of injury.\n\nIf severe weather approaches, switch off the device, move to a safe enclosed area, and avoid elevated or open locations.\n\nTeam PlayMetrics is not responsible for damage or injury caused by unsafe usage conditions.\n\nIntroduction',
 'Introduction\n\nThis manual describes the standard features and basic operation of the MMTT Hybrid Tracking Transceiver, developed by Te

In [7]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index(INDEX_NAME)

print(" Connected to Pinecone index:", INDEX_NAME)
print(" Index stats:", index.describe_index_stats())


 Connected to Pinecone index: mayihelp
 Index stats: {'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'rufus-data': {'vector_count': 86}},
 'total_vector_count': 86,
 'vector_type': 'dense'}


In [8]:
print(" Generating embeddings and upserting to Pinecone...")

all_vectors = []

for i in range(0, len(chunks), BATCH_SIZE):
    batch_chunks = chunks[i:i + BATCH_SIZE]

    embed_response = pc.inference.embed(
        model="llama-text-embed-v2",
        inputs=batch_chunks,
        parameters={
            "input_type": "passage"   # REQUIRED
        }
    )

    for j, emb in enumerate(embed_response):
        all_vectors.append({
            "id": f"chunk-{i + j}",
            "values": emb["values"],          # full 1024-dim vector
            "metadata": {"text": batch_chunks[j]}
        })

    index.upsert(
        vectors=all_vectors[-len(batch_chunks):],
        namespace=NAMESPACE
    )

print(" Upsert completed")
print(" Namespace stats:", index.describe_index_stats(namespace=NAMESPACE))


 Generating embeddings and upserting to Pinecone...
 Upsert completed
 Namespace stats: {'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'rufus-data': {'vector_count': 86}},
 'total_vector_count': 86,
 'vector_type': 'dense'}


In [9]:


# Correct & consistent index name
from pinecone import Pinecone
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

INDEX_NAME = "mayihelp"      # existing index
NAMESPACE = "rufus-data"

index = pc.Index(INDEX_NAME)
print(" Connected to index:", INDEX_NAME)


# Create index if it does not exist
if INDEX_NAME not in pc.list_indexes().names():
    print(" Creating Pinecone index...")

    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBED_DIM,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Connect to the index
index = pc.Index(INDEX_NAME)
print(" Pinecone index ready:", INDEX_NAME)

# Verify index
print(index.describe_index_stats())


 Connected to index: mayihelp
 Pinecone index ready: mayihelp
{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'rufus-data': {'vector_count': 86}},
 'total_vector_count': 86,
 'vector_type': 'dense'}


In [10]:
def retrieve_context(query, top_k=TOP_K):
    query_embedding = pc.inference.embed(
        model="llama-text-embed-v2",
        inputs=[query],
        parameters={
            "input_type": "query"   # REQUIRED
        }
    )[0]["values"]

    res = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        namespace=NAMESPACE
    )

    return [m["metadata"]["text"] for m in res["matches"]]


In [11]:
rag_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a website assistant. "
        "Answer ONLY using the provided context retrieved from the vector database. "
        "If the answer is not found in the context, say exactly: "
        "'NOt related to project or not asked properly.'"
    ),
    (
        "human",
        "Context:\n{context}\n\nQuestion:\n{question}"
    )
])


rag_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a website assistant. "
        "Answer ONLY using the provided context. "
        "If the answer is not found, say: "
        "'I don’t have enough information from the website to answer that.'"
    ),
    (
        "human",
        "Context:\n{context}\n\nQuestion:\n{question}"
    )
])

In [12]:
groq_client = Groq()

def ask_groq(context, question):
    messages = rag_prompt.format_messages(
        context=context[:MAX_CONTEXT_CHARS],
        question=question
    )

    groq_messages = []
    for m in messages:
        role = "user" if m.type == "human" else "system"
        groq_messages.append({
            "role": role,
            "content": m.content
        })

    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=groq_messages,
        temperature=0
    )

    return response.choices[0].message.content


In [13]:
def ask_website(question):
    retrieved_chunks = retrieve_context(question)
    context = "\n\n".join(retrieved_chunks)

    return ask_groq(context, question)


In [16]:
print(ask_website("car"))


NOt related to project or not asked properly.
